In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
import seaborn as sns
from scipy.stats import zscore
import math
from sklearn.impute import KNNImputer



In [3]:
data = pd.read_csv('customer.csv')

# The Unnamed: 0 column resulted from saving the csv file
data.drop('Unnamed: 0', axis=1, inplace=True) 
 
# Replace the null values in the 'is_employed' column with 'False' (empty means no employment)
data.fillna({'is_employed': 'False'}, inplace=True)

data['recent_move_b'] = data['recent_move_b'].apply(lambda x: True if x == 'T' else False)

data['is_employed'] = data['is_employed'].apply(lambda x: True if x == 'True' else False)

data.drop('custid', axis=1, inplace=True)

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72458 entries, 0 to 72457
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   sex             72458 non-null  object 
 1   is_employed     72458 non-null  bool   
 2   income          72458 non-null  float64
 3   marital_status  72458 non-null  object 
 4   health_ins      72458 non-null  bool   
 5   housing_type    70772 non-null  object 
 6   num_vehicles    70772 non-null  float64
 7   age             72458 non-null  int64  
 8   state_of_res    72458 non-null  object 
 9   code_column     72458 non-null  int64  
 10  gas_usage       70772 non-null  float64
 11  rooms           72458 non-null  int64  
 12  recent_move_b   72458 non-null  bool   
dtypes: bool(3), float64(3), int64(3), object(4)
memory usage: 5.7+ MB


In [4]:
# In the dictionary, we are indicated that the values 1, 2, 3 indicate special cases which we decided to separate
data['gas_payment'] = data['gas_usage'].replace([x for x in data['gas_usage'] if x not in [1, 2, 3]], 0)
data['gas_bill'] = data['gas_usage'].replace([x for x in data['gas_usage'] if x in [1, 2, 3]], 0)

data.drop('gas_usage', axis=1, inplace=True)

In [5]:
# Print the columns with missing values
print(f' --- PRE CLEANING --- \nColumns with missing values: \n{data.isna().sum()}')
old_len = len(data)
print('Length of the dataframe: ', old_len)   

# Drop rows with missing values except age column
data.dropna(inplace=True)


print(f'\n --- POST CLEANING --- \nLength of the dataframe: {len(data)}')
print('Number of rows dropped: ', old_len - len(data))

 --- PRE CLEANING --- 
Columns with missing values: 
sex                  0
is_employed          0
income               0
marital_status       0
health_ins           0
housing_type      1686
num_vehicles      1686
age                  0
state_of_res         0
code_column          0
rooms                0
recent_move_b        0
gas_payment          0
gas_bill          1686
dtype: int64
Length of the dataframe:  72458

 --- POST CLEANING --- 
Length of the dataframe: 70772
Number of rows dropped:  1686


In [6]:
data['age'] = data['age'].replace(0, np.nan)

In [7]:
scaling_cols = ['income', 'gas_payment']

scaling_df = data[scaling_cols]


# Dataset with log scaling
data_log = data.copy()
data_log[scaling_cols] = np.log1p(scaling_df)

# Dataset with zscore scaling
data_zs = data.copy()
data_zs[scaling_cols] = scaling_df.apply(zscore)

data_mm = data.copy()

dfs = [data_mm, data_log, data_zs]

for i, df in enumerate(dfs):
    numerical_cols = df.select_dtypes(include=[np.number])
    numerical_cols.drop('age', axis=1, inplace=True)
    cols = (numerical_cols-numerical_cols.min())/(numerical_cols.max()-numerical_cols.min())
    df[numerical_cols.columns] = cols
    dfs[i] = df


dfs = [data] + dfs

c:\Users\castr\anaconda3\envs\IAP2\Lib\site-packages\pandas\core\internals\blocks.py:393: RuntimeWarning: invalid value encountered in log1p
  result = func(self.values, **kwargs)


In [8]:
from sklearn.preprocessing import OneHotEncoder

encoders = []

for i, df in enumerate(dfs):
    categorical_cols = df.select_dtypes(include=[object])
    if categorical_cols.empty:
        continue

    encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    encoded_array = encoder.fit_transform(categorical_cols)
    encoded_df = pd.DataFrame(
        encoded_array,
        columns=encoder.get_feature_names_out(categorical_cols.columns),
        index=df.index
    )
    
    df.drop(columns=categorical_cols.columns, inplace=True)
    df = pd.concat([df, encoded_df], axis=1)
    encoders.append(encoder)
    dfs[i] = df

# Drop null values (income is null if it was negative because of the log scaling)
dfs[2] = dfs[2].dropna(axis=1)

In [9]:
dfs[3].head()

,is_employed,income,health_ins,num_vehicles,age,code_column,rooms,recent_move_b,gas_payment,gas_bill,...,state_of_res_South Dakota,state_of_res_Tennessee,state_of_res_Texas,state_of_res_Utah,state_of_res_Vermont,state_of_res_Virginia,state_of_res_Washington,state_of_res_West Virginia,state_of_res_Wisconsin,state_of_res_Wyoming
0,False,0.022866,True,0.000000,24.0,0.103726,0.4,False,0.0,0.368421,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,False,0.023815,True,0.000000,82.0,0.103726,1.0,True,1.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,False,0.022075,True,0.333333,31.0,0.103726,0.4,False,0.0,0.070175,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,False,0.035343,True,0.166667,93.0,0.103726,0.2,False,0.0,0.210526,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,False,0.036316,True,0.333333,67.0,0.103726,0.2,False,1.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
dfs[2].head()

,is_employed,health_ins,num_vehicles,code_column,rooms,recent_move_b,gas_payment,gas_bill,sex_Female,sex_Male,...,state_of_res_South Dakota,state_of_res_Tennessee,state_of_res_Texas,state_of_res_Utah,state_of_res_Vermont,state_of_res_Virginia,state_of_res_Washington,state_of_res_West Virginia,state_of_res_Wisconsin,state_of_res_Wyoming
0,False,True,0.000000,0.103726,0.4,False,0.0,0.368421,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,False,True,0.000000,0.103726,1.0,True,1.0,0.000000,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,False,True,0.333333,0.103726,0.4,False,0.0,0.070175,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,False,True,0.166667,0.103726,0.2,False,0.0,0.210526,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,False,True,0.333333,0.103726,0.2,False,1.0,0.000000,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
for i, df in enumerate(dfs):
    imputer = KNNImputer(n_neighbors=5)
    imputed_data = imputer.fit_transform(df)
    age = np.round(imputed_data[:, 4]) # Rounds to the nearest integer
    df['age'] = (age - age.min())/(age.max()-age.min()) # Min-Max scaling
    dfs[i] = df

In [12]:
dfs[2][dfs[2].isna().any(axis=1)]

,is_employed,health_ins,num_vehicles,code_column,rooms,recent_move_b,gas_payment,gas_bill,sex_Female,sex_Male,...,state_of_res_Tennessee,state_of_res_Texas,state_of_res_Utah,state_of_res_Vermont,state_of_res_Virginia,state_of_res_Washington,state_of_res_West Virginia,state_of_res_Wisconsin,state_of_res_Wyoming,age


In [13]:
# save all dfs processed in csv
for i, df in enumerate(dfs):
    df.to_csv(f'processed_df_{i}.csv', index=False)

## Algorithms

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix




# Initialize models to be evaluated
models = {
    'Logistic Regression': LogisticRegression(),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier()
}

# Load the processed DataFrames
dfs = [pd.read_csv(f'dfs_processed/processed_df_{i}.csv') for i in range(4)]


dfs1 = dfs


# Loop through the list of DataFrames
for idx, data in enumerate(dfs1):
    print(f"\nProcessing dataset {idx}...")
    
    # Assuming the target column is named 'target', adjust as necessary
    X = data.drop(columns=['health_ins'])
    y = data['health_ins']

    print(X.isna().sum())
    
    # Split into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    # Standardize the features (optional, good practice for SVM, Logistic Regression)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # Evaluate each model
    for model_name, model in models.items():
        print(f"  Training {model_name}...")
        model.fit(X_train, y_train)
        print()
        y_pred = model.predict(X_test)
        print(classification_report(y_test, y_pred, digits=6))

        print(confusion_matrix(y_test, y_pred))



Processing dataset 0...
is_employed                   0
income                        0
num_vehicles                  0
age                           0
code_column                   0
                             ..
state_of_res_Virginia         0
state_of_res_Washington       0
state_of_res_West Virginia    0
state_of_res_Wisconsin        0
state_of_res_Wyoming          0
Length: 70, dtype: int64
  Training Logistic Regression...

              precision    recall  f1-score   support

       False   0.375000  0.009419  0.018377      1911
        True   0.910640  0.998447  0.952524     19321

    accuracy                       0.909429     21232
   macro avg   0.642820  0.503933  0.485451     21232
weighted avg   0.862429  0.909429  0.868446     21232

[[   18  1893]
 [   30 19291]]
  Training Decision Tree...

              precision    recall  f1-score   support

       False   0.221724  0.232862  0.227157      1911
        True   0.923745  0.919155  0.921445     19321

    accuracy

c:\Users\castr\anaconda3\envs\IAP2\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\castr\anaconda3\envs\IAP2\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\castr\anaconda3\envs\IAP2\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



              precision    recall  f1-score   support

       False   0.152924  0.166928  0.159620      1911
        True   0.916849  0.908545  0.912678     19321

    accuracy                       0.841795     21232
   macro avg   0.534887  0.537737  0.536149     21232
weighted avg   0.848092  0.841795  0.844899     21232

[[  319  1592]
 [ 1767 17554]]
  Training Random Forest...

              precision    recall  f1-score   support

       False   0.172360  0.058085  0.086888      1911
        True   0.912570  0.972413  0.941542     19321

    accuracy                       0.890119     21232
   macro avg   0.542465  0.515249  0.514215     21232
weighted avg   0.845947  0.890119  0.864618     21232

[[  111  1800]
 [  533 18788]]

Processing dataset 3...
is_employed                   0
income                        0
num_vehicles                  0
age                           0
code_column                   0
                             ..
state_of_res_Virginia         0
state

In [ ]:




from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split
import pandas as pd

# Load the processed DataFrames
dfs = [pd.read_csv(f'dfs_processed/processed_df_{i}.csv') for i in range(4)]

dfs1 = dfs

# Loop through the list of DataFrames
for idx, data in enumerate(dfs1):
    print(f"\nProcessing dataset {idx}...")
    
    # Assuming the target column is named 'health_ins'
    X = data.drop(columns=['health_ins'])
    y = data['health_ins']
    
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    # Do not scale the data
    clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
    
    models, predictions = clf.fit(X_train, X_test, y_train, y_test)
    print(models)



Processing dataset 0...


 38%|███▊      | 11/29 [00:18<00:43,  2.40s/it]